In [6]:
import pandas as pd
import sqlite3

csvfile = 'cran_logs_2015_01_01.csv'

con = sqlite3.connect("day9.db")

sql_lines = pd.read_csv(csvfile, delimiter=",").to_sql('cran_logs', con, if_exists='replace', index=False)
print("Imported SQL lines:", sql_lines)

cur = con.cursor()

Imported SQL lines:  62324


# cran_logs columns
- download_date
- time
- size
- r_version
- r_arch
- r_os
- package
- version
- country
- ip_id

In [16]:
# -- 9.1 Give the package name and how many times they're downloaded. Order by the 2nd column descently.
cur.execute("""
    SELECT package, COUNT(package)
    FROM cran_logs
    GROUP BY package
    ORDER BY COUNT(package) DESC
    LIMIT 10
""")
display(cur.fetchall())

[('digest', 1009),
 ('DBI', 973),
 ('plyr', 896),
 ('manipulate', 877),
 ('Rcpp', 867),
 ('ggplot2', 817),
 ('RColorBrewer', 746),
 ('reshape2', 717),
 ('rJava', 641),
 ('stringr', 585)]

In [20]:
# -- 9.2 Give the package ranking (based on how many times it was downloaded) during 9AM to 11AM
cur.execute("""
    SELECT
        package,
        COUNT(package),
        RANK() OVER (ORDER BY  COUNT(package) DESC) AS rank
    FROM cran_logs
    WHERE `time` >= '09:00:00' AND `time` <= '11:00:00'
    GROUP BY package
    ORDER BY COUNT(package) DESC
    LIMIT 10
""")
display(cur.fetchall())

[('DBI', 83, 1),
 ('ggplot2', 74, 2),
 ('digest', 73, 3),
 ('reshape2', 67, 4),
 ('RColorBrewer', 66, 5),
 ('plyr', 61, 6),
 ('colorspace', 61, 6),
 ('Rcpp', 59, 8),
 ('scales', 58, 9),
 ('munsell', 56, 10)]

In [22]:
# -- 9.3 How many recordings are from China ("CN") or Japan("JP") or Singapore ("SG")?
cur.execute("""
    SELECT
        country,
        COUNT(country)
    FROM cran_logs
    WHERE country IN ('CN', 'JP', 'SG')
    GROUP BY country
    ORDER BY COUNT(country) DESC
    LIMIT 10
""")
display(cur.fetchall())

[('CN', 6424), ('JP', 1887), ('SG', 481)]

In [28]:
# -- 9.4 Print the countries whose downloaded are more than the downloads from China ("CN")
cur.execute("""
    SELECT
        country,
        COUNT(country) AS cnt
    FROM cran_logs
    GROUP BY country
    HAVING cnt > (SELECT COUNT(*) FROM cran_logs WHERE country = 'CN')
    ORDER BY COUNT(country) DESC
""")
display(cur.fetchall())

[('US', 20113), ('FR', 6504)]

In [31]:
# -- 9.5 Print the average length of the package name of all the UNIQUE packages
cur.execute("""
    SELECT
        AVG(LENGTH(DISTINCT package))
    FROM cran_logs
""")
display(cur.fetchone()[0])

6.712950322031448

In [34]:
# -- 9.6 Get the package whose download count ranks 2nd (print package name and its download count).
cur.execute("""
    SELECT package, COUNT(package)
    FROM cran_logs
    GROUP BY package
    ORDER BY COUNT(package) DESC
    LIMIT 1, 1
""")
display(cur.fetchall())

[('DBI', 973)]

In [36]:
# -- 9.7 Print the name of the package whose download count is bigger than 1000.
cur.execute("""
    SELECT package, COUNT(package)
    FROM cran_logs
    GROUP BY package
    HAVING COUNT(package) > 1000
    ORDER BY COUNT(package) DESC
""")
display(cur.fetchall())

[('digest', 1009)]

In [43]:
# -- 9.8 The field "r_os" is the operating system of the users.
#    -- 	Here we would like to know what main system we have (ignore version number), the relevant counts, and the proportion (in percentage).
cur.execute("""
    SELECT
        r_os,
        COUNT(r_os) AS cnt,
        100 * COUNT(r_os)/SUM(COUNT(r_os)) OVER (PARTITION BY r_os) AS share
    FROM cran_logs
    GROUP BY r_os
    ORDER BY cnt DESC
""")
display(cur.fetchall())

[('mingw32', 28994, 100),
 ('linux-gnu', 15487, 100),
 ('darwin13.4.0', 3098, 100),
 ('darwin10.8.0', 2527, 100),
 ('darwin13.1.0', 1675, 100),
 ('darwin9.8.0', 292, 100),
 ('darwin14.0.0', 231, 100),
 ('darwin14.1.0', 23, 100),
 ('darwin13.3.0', 12, 100),
 ('linux-gnueabihf', 5, 100),
 ('darwin13.2.0', 2, 100),
 (None, 0, None)]